# Tensile: Enhancing agent reliability through automated test synthesis

## Summary

Tensile is DataRobot's proprietary test-driven development framework designed to systematically improve the reliability, task performance, and policy adherence of AI agents. It aims to move agent development beyond "vibe coding" by providing a quantitative, repeatable way to diagnose and remediate errors in complex, multi-turn agentic workflows. The framework operates through a three-step **Enhanced Agent Improvement Cycle** that begins by instrumenting an agent to record its execution trajectories.

The core of Tensile lies in its ability to identify **testable moments**—specific points in a recorded trajectory where an agent either succeeded or failed (e.g., making a hallucinated tool call or violating a safety policy). Once identified, these moments are synthesized into short, reproducible tests that can be replayed to quantify performance improvements. Remediation is then achieved through several methods, including manual system prompt or tool updates, and **Contextual Hints**—surgically accurate messages injected at runtime to steer agent behavior.

This accelerator walks through:

1. Instrumenting an agent for trajectory logging
2. Running the full analysis pipeline
3. Evaluating testable moments
4. Replaying trajectories
5. Configuration with the DataRobot LLM Gateway
6. Clustering (exploration app and hint injector)
7. The Trajectory Analyzer workflow for iterative improvement

## Prerequisites

- Python 3.13 (recommended)
- [uv](https://docs.astral.sh/uv/) for environment and dependency management
- A Tensile installation (e.g. from source: `uv pip install -e .` in the Tensile repo)
- A `config.yaml` (copy from `config.yaml.sample` and fill in credentials)
- For DataRobot LLM Gateway: API token and endpoint URL for your region

## Quickstart

From the Tensile project root:

```bash
uv venv --python 3.13
uv sync; pre-commit install
uv pip install -e .
cp config.yaml.sample config.yaml   # Fill in credentials
tensile   # Show help
```

Trajectories are logged to `<trajectory_dir>/<subdir>` as defined in `config.yaml`.

## Setup

Import dependencies and load environment variables. Ensure your Tensile environment is activated and `config.yaml` is configured.

In [ ]:
import os
from pathlib import Path

import httpx
from dotenv import load_dotenv
from openai import AsyncOpenAI

load_dotenv(override=True)

# Credentials for LLM (e.g. DataRobot API token when using LLM Gateway)
api_key = os.getenv("DATAROBOT_API_TOKEN", "")
endpoint_url = os.getenv("DATAROBOT_LLM_GATEWAY_URL", "https://app.datarobot.com/api/v2/genai/llmgw")

## 1. Instrument an agent for trajectory logging

Use `TrajectoryLogger` as an httpx transport wrapper so that every agent run is recorded. Then pass the resulting `http_client` into your OpenAI-compatible client (e.g. DataRobot LLM Gateway).

In [ ]:
# Run this cell only if Tensile is installed (e.g. uv pip install -e . in Tensile repo)
from tensile.logging import TrajectoryLogger

http_client = httpx.AsyncClient(
    transport=TrajectoryLogger(
        httpx.AsyncHTTPTransport(),
        trajectory_subdir="my_agent",  # or None for default subdir
    )
)

client = AsyncOpenAI(
    api_key=api_key,
    base_url=f"{endpoint_url}/v1",
    http_client=http_client,
)

# Use client for chat completions; trajectories will be logged to
# <trajectory_dir>/my_agent/ as defined in config.yaml

## 2. Run the full analysis pipeline

After running your agent and generating trajectory files, analyze a trajectory to identify testable moments and produce hints. Outputs are written to `analysis_output/` by default.

**CLI (run in terminal):**

```bash
tensile analyze trajectories/taco/sde-implement-raft-in-go.jsonl
```

Replace the path with your own trajectory file (e.g. under `<trajectory_dir>/<subdir>/`).

## 3. Evaluate testable moments

Run specific testable moments multiple times to measure consistency and regression.

**CLI:**

```bash
tensile test <moment_path> -n 10
```

## 4. Replay trajectories

Replay each step in a trajectory multiple times to collect new LLM responses for comparison with the original. This helps identify flukes (unlikely actions that disappear on replay) and understand how changes (e.g. system prompt updates) affect agent behavior.

**CLI examples:**

```bash
# Basic replay (1 replay per step)
tensile replay <trajectory_file> [output_path]

# Replay each step multiple times
tensile replay <trajectory_file> --num-replays 5

# Control concurrency (default: 5)
tensile replay <trajectory_file> --num-replays 3 --max-concurrency 10

# Replay with a new system prompt
tensile replay <trajectory_file> --num-replays 3 --system-prompt-path <system_prompt_path_txt>
```

If `output_path` is omitted, output is written to `<trajectory_file>.replay.jsonl`.

## Configuration

### LLM configuration with DataRobot LLM Gateway

To use the DataRobot LLM Gateway with Tensile, add the following to your `config.yaml`:

```yaml
# config.yaml
llm:
  name: "vertex_ai/gemini-3-pro-preview"   # or another model of your choice
  api_base: "https://app.datarobot.com/api/v2/genai/llmgw"
  api_key: <your datarobot api token>
```

Other OpenAI-compatible endpoints can be configured the same way.

## Clustering

### Clustering app

Tensile includes a Dash app to explore and cluster analysis outputs and messages.

**Start the app (from Tensile project):**

```bash
task apps:clustering
```

Install dev dependencies if needed: `task dev-env` (or with uv: install the `dev` dependency group).

### Clustering-based hint injector

Use the clustering-based hint injector to surface past analyses and successful answers inside live LLM calls. Wrap your httpx transport with `ClusteringHintInjector` and point it at your Tensile outputs (`analysis_output/` and `trajectories/` by default).

In [ ]:
# Example: wire ClusteringHintInjector for an async OpenAI-compatible client
# Requires Tensile installed and sentence-transformers for embeddings

from tensile.logging.hint_injector import (
    ClusteringHintConfig,
    ClusteringHintInjector,
    InMemoryReportStore,
    SentenceTransformersEmbeddingBackend,
)

base_transport = httpx.AsyncHTTPTransport()
embedding_backend = SentenceTransformersEmbeddingBackend(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
)
report_store = InMemoryReportStore()
config = ClusteringHintConfig(
    analysis_dirs=[Path("analysis_output")],
    trajectories_dirs=[Path("trajectories")],
)

hinting_transport = ClusteringHintInjector(
    base_transport,
    embedding_backend=embedding_backend,
    report_store=report_store,
    config=config,
)

http_client = httpx.AsyncClient(transport=hinting_transport)
client = AsyncOpenAI(
    api_key=api_key,
    base_url=f"{endpoint_url}/v1",
    http_client=http_client,
)

## Trajectory Analyzer workflow

End-to-end loop for iterative agent improvement using programmatic hints and trajectory analysis:

1. **Instrument** the agent with `ProgrammaticHintInjector` and `TrajectoryLogger` (start with `hint_file_path=None` until you have a hint file).
2. **Run the agent** to generate a trajectory.
3. **Run** `tensile analyze <trajectory_path>`; copy the resulting `hints.json`, updated system prompt, and/or tool definitions back into your agent.
4. Set `hint_file_path` to the `hints.json` file and **rerun** the agent to produce a new trajectory.
5. **Re-analyze** with `tensile analyze <new_traj_path> --hints-file <path_to_hints.json>`.
6. **Repeat** until behavior converges.

In [ ]:
from tensile.logging import TrajectoryLogger
from tensile.logging.hint_injector.programmatic_hint_injector import ProgrammaticHintInjector

http_client = httpx.AsyncClient(
    transport=ProgrammaticHintInjector(
        wrapped=TrajectoryLogger(
            wrapped=httpx.AsyncHTTPTransport(),
            trajectory_subdir="my_agent",
        ),
        hint_file_path=None,  # Set to path to hints.json after first analysis
    )
)

client = AsyncOpenAI(
    api_key=api_key,
    base_url=f"{endpoint_url}/v1",
    http_client=http_client,
)